In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from machineLearning_helper import load_features_data, normalize_train_X, normalize_test_X
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt 
from confusion_matrix import create_confusion_matrix

In [2]:
# loading feature data

cobbleStone_data = load_features_data('../../features/cobbleStone.csv')
curbStone_data = load_features_data('../../features/curbStone.csv')

# print('Cobble Stone: ', cobbleStone_data.shape)
# print('Curb Stone: ', curbStone_data.shape)

In [3]:
# selecting and labelling data 

cobbleStone = cobbleStone_data.loc[cobbleStone_data['Object'] == 'Cobble Stone'].iloc[:,5:]
curbStone = curbStone_data.loc[curbStone_data['Object'] == 'Curb Stone'].iloc[:, 5:]

cobbleStone_label = ['COBBLESTONE']*cobbleStone.shape[0]
curbStone_label = ['CURBSTONE']*curbStone.shape[0]

data = cobbleStone.values.tolist() + curbStone.values.tolist()
label = cobbleStone_label + curbStone_label
print(np.shape(data))
print(np.shape(label))

(49981, 42)
(49981,)


In [4]:
#splitting data for training and testing

train_X, test_X, train_y, test_y = train_test_split(data, label, test_size=0.33, random_state=42)
# print('X train data:', np.array(train_X).shape)
# print('X test data:', np.array(test_X).shape)
# print('y train data:', np.array(train_y).shape)
# print('y test data:', np.array(test_y).shape)

In [7]:
normalized_X_train = normalize_train_X(train_X)
normalized_X_test = normalize_test_X(test_X)

In [ ]:
#Random forest classifier

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)
clf.fit(normalized_X_train, train_y)
result = clf.predict(normalized_X_test)

cm = confusion_matrix(test_y, result)
print(cm)
print("Accuracy: ",accuracy_score(test_y, result))
labels = ['COBBLESTONE', 'CURBSTONE']
title = ('Random Forest Confusion Matrix')
create_confusion_matrix(test_y, result, labels, title)

In [ ]:
#KNN

from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=2)
clf.fit(normalized_X_train, train_y)
result = clf.predict(normalized_X_test)

cm = confusion_matrix(test_y, result)
print(cm)
print("Accuracy: ",accuracy_score(test_y, result))
labels = ['COBBLESTONE', 'CURBSTONE']
title = ('KNN Confusion Matrix')
create_confusion_matrix(test_y, result, labels, title)

In [ ]:
#reshaping the features into 6x6 matrix for CNN
train_X = np.array(normalized_X_train)[:,4:40].reshape(-1,6,6,1)
test_X =  np.array(normalized_X_test)[:,4:40].reshape(-1,6,6,1)

train_X.shape, test_X.shape

In [ ]:
#converting training objects into classes
classes = np.unique(train_y)
nClasses = len(classes)
# print('Total number of outputs : ', nClasses)
# print('Output classes : ', classes)

In [ ]:
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')

train_X.shape, test_X.shape, train_y.shape, test_y.shape

In [ ]:
from tensorflow.keras.utils import to_categorical

# Change the labels from categorical to one-hot encoding
train_Y_one_hot = keras.utils.to_categorical(train_y)
test_Y_one_hot = keras.utils.to_categorical(test_y)

# Display the change for category label using one-hot encoding
print('Original label:', train_y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

print(train_Y_one_hot.shape)
print(test_Y_one_hot.shape)

In [ ]:
#splitting the training data set for CNN

train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.30, random_state=13)
train_X.shape,valid_X.shape,train_label.shape,valid_label.shape

In [ ]:
# hidden layers in CNN

epochs = 20
num_classes = 2
batch_size= 100

model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), padding='same', input_shape=(6,6,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))

model.add(Flatten())
model.add(Dense(num_classes, Activation('softmax')))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])